In [1]:
!pip install -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -U datasets bitsandbytes einops wandb
!pip install fastapi nest-asyncio pyngrok uvicorn

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-5n8of43v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-5n8of43v
  Resolved https://github.com/huggingface/peft.git to commit 8665e2b5719faa4e4b91749ddec09442927b53e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for trl from https://files.pythonhosted.org/packages/e8/c2/4c27977024023c762564ab74f8ac39cbf04b8e886300cde1f5a59a5e9573/trl-0.7.7-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.9 MB/s eta 0:00:00
  Obtaining dependency information for tyro>=0.5.11 from https://files.python

In [2]:
#!zip -ry /kaggle/working/sample /kaggle/working/clean-dataset-for-pretraining/sample
%cp /kaggle/input/pretraining-clean-dataset/data/sample.mp4 /kaggle/working/
%mv sample.mp4 sample.zip
!ls

sample.zip


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

import gc
gc.collect()
torch.cuda.empty_cache()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#model_name = "ybelkada/falcon-7b-sharded-bf16"
model_name = "microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)
#model.config.use_cache = False
#model = model.to(device)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer_length = len(tokenizer)
seq_length = 512
#from transformers import pipeline
#gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=300)


api_busy = False
def generate_text(prompt):
    global api_busy
    if api_busy:
        return None
    else:
        api_busy = True
        '''input_ids = tokenizer.encode(prompt).ids
        #padding_tokens_count = seq_length - len(input_ids)
        input_tokens = torch.stack(
            [
                torch.tensor(input_ids),
                torch.tensor([tokenizer.pad_token] * padding_tokens_count, dtype=torch.int64)
            ],
            dim=0
        )'''
        
        with torch.no_grad():
            token_ids = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")
            output_ids = model.generate(
                token_ids.to(model.device),
                max_new_tokens=seq_length,
                temperature=0.3,
                do_sample=True
            )
            output = tokenizer.decode(output_ids[0])
            resp = output
        #result = gen(prompt)
        #resp = result[0]['generated_text']
        api_busy = False
    return resp

cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#generate_text("opus")

In [5]:
import zipfile
import random, json
file_list = []
dataset_path = '/kaggle/working/sample.zip'
with zipfile.ZipFile(dataset_path, mode="r") as archive:
    for a in archive.infolist():
        count = 0
        if '__MACOSX' in a.filename:
            continue
        if '.jsonl' in a.filename and a.file_size > 0:
            file_list.append( (a.filename, a.file_size) )
    
def read_text():
    fidx = random.randint(0, len(file_list)-1)
    file_name = file_list[fidx][0]
    with zipfile.ZipFile(dataset_path, mode="r") as archive:
        with archive.open(file_name) as f:
            d = list(f)
            json_len = len(d)
            idx = random.randint(0, json_len-1 )
            data = json.loads(d[idx])
            #data = d[idx]
            #print(data['text'])
            return data['text']
        


In [ ]:
#read_text()

In [6]:
import asyncio
import os, random
loop = asyncio.get_event_loop()
import queue
q = queue.Queue()

def callback(flag=False):
    print("Adding Text To Queue", flag)
    if flag:
        idx = random.randint(1, tokenizer_length)
        token = tokenizer.decode([idx])
        q.put( generate_text(token) )
        #q.put( read_text() )
    else:
        q.put( read_text() )
    
    
    if q.qsize() > 100:
        flag = True
    else:
        flag = False
    
    loop.call_later(0.01, callback, flag)
    #callback()
    
            
callback()
#loop.call_later(1, callback, True)

Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False


In [7]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import random
app = FastAPI()

# middlewares
app.add_middleware(
    CORSMiddleware, # https://fastapi.tiangolo.com/tutorial/cors/
    allow_origins=['*'], # wildcard to allow all, more here - https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Origin
    allow_credentials=True, # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Credentials
    allow_methods=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Methods
    allow_headers=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Headers
)

@app.get('/generate')
async def generate():
    global q
    count = 4
    resp_arr = []
    for i in range(count):
        '''if random.random() < 0.5:
            resp_arr.append( q.get() )
            q.task_done()
            
        else:
            resp_arr.append( read_text() )'''
        
        resp_arr.append( q.get() )
        q.task_done()
            
    return resp_arr


@app.get('/')
async def root():
    return {'hello': 'world'}

Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False


In [ ]:
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn


Ngrok_token = "2aA4bHBrJQJs9ugxgp9xWa00tkL_7iNEa98wY17WQmYrK6ELm" 

# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token))

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)

INFO:     Started server process [127]
INFO:     Waiting for application startup.


Public URL: https://e60f-35-233-217-40.ngrok-free.app
Adding Text To Queue False


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
Adding Text To Queue False
A

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Adding Text To Queue True
Adding Text To Queue True
Adding Text To Queue True
Adding Text To Queue True
Adding Text To Queue True
Adding Text To Queue True
Adding Text To Queue True
Adding Text To Queue True
Adding Text To Queue True
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue True
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue True
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue True
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue True
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue False
Adding Text To Queue False
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue False
Adding Text To Queue False
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue False
Adding Text To Queue False
INFO:     35.204.56.236:0 - "GET /generate HTTP/1.1" 200 OK
Adding Text To Queue

In [2]:
loop.stop()

NameError: name 'loop' is not defined